In [91]:
from cyvcf2 import VCF
from collections import Counter
import pandas as pd
import numpy as np

In [51]:
## load file into notebook
infile = "../../../stripes_AIL_gg6a/F15/stripes_input/all_f15_only_founder_snps.vcf.bgz"
vcf = VCF(infile, threads=5)

In [52]:
#get sample_names
sample = vcf.samples
sampl = [i.rsplit("/",1)[1].split("_")[1] for i in sample] # adjust this if naming convention of files differs

In [54]:
len(sample)

977

In [55]:
sample_ids = []
for s in sample:
    s2 = s.rsplit("/",1)[1]
    #print(s2)
    s3 = s2.split("_")
    if s3[0] == "F15": # ugh, why dont all files follow the same naming convention.
        #print(s3[0])
        sample_ids.append(s3[1])
    else:
        #print(s3[1])
        sample_ids.append(s3[0])        

In [63]:
#Counter(sample_ids).most_common(55)

In [59]:
d=dict(Counter(sample_ids))
#get all samples that are duplicates
dups = [key for key, item in d.items() if item>1]

In [64]:
#dups


In [62]:
#check if fastq files for both exist
for duplicate in dups:
    print(duplicate)
    !find /mnt/bbg/AIL_reseq/ -name *{duplicate}* #ipython, because i cant be bothered to use os right now

2471
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/merged/F15_2471_Sx_merged_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/merged/F15_2471_Sx_merged_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/to.merge/fc2/F15_2471_S9_L005_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/to.merge/fc2/F15_2471_S9_L005_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/to.merge/fc3/F15_2471_S44_L008_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/to.merge/fc3/F15_2471_S44_L008_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/all/152471_forward.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/all/152471_reverse.fastq.gz
2811
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/fc2/F15_2811_S77_L006_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/fc2/F15_2811_S77_L006_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/testplate/2811_F15_S74_L005_R1_001.fastq.gz
/mnt/

In [65]:

# run vcftools to check which plate/copy was better in missingness:

! vcftools --missing-indv  --vcf {infile}


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf ../../../stripes_AIL_gg6a/F15/stripes_input/all_f15_only_founder_snps.vcf.bgz
	--missing-indv

After filtering, kept 977 out of 977 Individuals
Outputting Individual Missingness
After filtering, kept 8178314 out of a possible 8178314 Sites
Run Time = 876.00 seconds


In [72]:
#! mv out.log  ../../../stripes_AIL_gg6a/F15/stripes_input/out.log

In [73]:
with open("../../../stripes_AIL_gg6a/F15/stripes_input/out.imiss", "r") as handle: 
    qual = handle.read().split("\n")
    qual.pop(0)
    qual = [i.split("\t") for i in qual]     

In [93]:
#check which samples are bad:
p = {}
for i in qual:     
    if len(i)>1:
        s = i[0].rsplit("/",1)[1].split("_")[0]
        s2 = i[0].rsplit("/",1)[1].split("_")[1]
        if s in dups:
            p.setdefault(s, {})[i[4]] = str(i[0])
        elif s2 in dups:
            p.setdefault(s2, {})[i[4]] = str(i[0])
            
            #print(i[0].rsplit("/",1)[1].split("_")[0])
            #print(i[0]) 
            #print(i[4]) # fraction of called variants 


In [94]:
len(p.keys())

50

In [108]:
best_per_sample = []
for key, item in p.items():
    all_miss = np.array([float(i) for i in list(item.keys())])
    print(all_miss)
    print(np.max(all_miss))
    print(item[str(np.max(all_miss))])
    best_per_sample.append(item[str(np.max(all_miss))])        

[0.914441 0.557073 0.606762]
0.914441
/home/tilman/storage2/F15/data/F15_2471_S9_L005_call2.bam
[0.734657 0.451385]
0.734657
/home/tilman/storage2/F15/data/F15_2811_S77_L006_call2.bam
[0.995032 0.592368 0.595165]
0.995032
/home/tilman/storage2/F15/data/F15_2478_S13_L005_call2.bam
[0.727882 0.515682]
0.727882
/home/tilman/storage2/F15/data/F15_2373_S30_L008_call2.bam
[0.698232 0.673859 0.963907]
0.963907
/home/tilman/storage2/F15/data/F15_3054_S37_L006_call2.bam
[0.463385 0.647414]
0.647414
/home/tilman/storage2/F15/data/F15_2753_S60_L007_call2.bam
[0.783136 0.48085 ]
0.783136
/home/tilman/storage2/F15/data/F15_2813_S78_L006_call2.bam
[0.665974 0.732037 0.907234]
0.907234
/home/tilman/storage2/F15/data/F15_3293_S31_L008_call2.bam
[0.712453 0.979047 0.727227]
0.979047
/home/tilman/storage2/F15/data/F15_3165_S25_L007_call2.bam
[0.985668 0.818109]
0.985668
/home/tilman/storage2/F15/data/2427_F15_S63_L005_call2.bam
[0.492446 0.992901]
0.992901
/home/tilman/storage2/F15/data/F15_2432_S78_L00

In [117]:
filter_out = []
for key, item in p.items():
    for k, i in item.items():
        if not i in best_per_sample:
            filter_out.append(i)

In [119]:
#print(filter_out)

In [120]:
## kicking out:
exclude_samples = filter_out
remove_samples = "../../../stripes_AIL_gg6a/F15/stripes_input/remove_F15.txt"

with open(remove_samples, "w") as handle:
    handle.write("\n".join(exclude_samples))

In [121]:
output = infile.split(".vcf")[0]+"_removed_dups.vcf"
print(output)

../../../stripes_AIL_gg6a/F15/stripes_input/all_f15_only_founder_snps_removed_dups.vcf


In [122]:
## filtering out the bad duplicate samples using bcftools

In [123]:
! bcftools view --samples-file ^{remove_samples} -Ov {infile} -o {output}

In [124]:
vcf2 = VCF(output, threads=4)

In [126]:
print(len(vcf.samples))
print(len(vcf2.samples))  #cool, i actually filtered them out.
samples = vcf2.samples 

977
908


In [127]:
sample_ids = []
for s in samples:
    s2 = s.rsplit("/",1)[1]
    #print(s2)
    s3 = s2.split("_")
    if s3[0] == "F15": # ugh, why dont all files follow the same naming convention.
        #print(s3[0])
        sample_ids.append(s3[1]+"15")
    else:
        #print(s3[1])
        sample_ids.append(s3[0]+"15")        

In [128]:
Counter(sample_ids).most_common(5)  ## cool, no duplicates.

[('239815', 1), ('324815', 1), ('307915', 1), ('240615', 1), ('292115', 1)]

In [129]:
new_samples = infile.rsplit("/",1)[0]+"/new_names_F15.txt"

In [130]:
new_samples

'../../../stripes_AIL_gg6a/F15/stripes_input/new_names_F15.txt'

In [132]:
with open(new_samples, "w") as handle:
    handle.write("\n".join(sample_ids))

In [133]:
output2 = output.rsplit(".",1)[0]+"_new_names.vcf"

In [134]:
! bcftools reheader --samples {new_samples} -o {output2} {output}  ###rename samples

In [135]:
!grep "#CHROM" {output2} #yay

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	239815	324815	307915	240615	292115	314615	247115	318715	303515	257415	240315	323015	281115	331515	312115	247815	326915	237315	317715	818315	281315	832815	276015	830915	327615	291415	295715	237815	240015	328915	286915	304315	824315	274515	319715	242615	272815	311515	242715	835115	811715	301315	294715	295415	286315	313615	317015	295315	824515	332615	252415	235315	244815	245815	289215	298815	253315	326815	237215	256815	260815	294215	312515	234415	323115	300415	332515	254415	282615	823115	258715	313315	253915	261915	280615	300815	820515	867515	250815	328615	276115	322215	259915	233515	330415	328115	296115	271315	312015	247415	316815	246015	310915	239315	260415	271515	238215	248015	325915	325115	328015	829715	305115	233815	299215	288515	322615	246815	314415	255615	284715	829315	282315	280815	311215	297615	291815	254715	823715	302815	283815	284815	318515	286415	239115	236315	254215	273315	288415	296815	819315	249015	314815	250115	283515	297215	24

In [50]:
##now change the vcf and the sample-file for stripes & we're set.